# 환경 준비

## 라이브러리 임포트

In [ ]:
import os
from typing import Tuple, List, Sequence, Callable

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision.models as models

!pip install facenet-pytorch
from facenet_pytorch import InceptionResnetV1

!pip install torch_optimizer
import torch_optimizer as optim

!pip install -U git+https://github.com/albu/albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F
from sklearn.model_selection import train_test_split, KFold

import time
import random

## 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
os.chdir('/content/drive/MyDrive/statml_competition/')

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

print('using device:', device)

In [ ]:
device = "cuda:0"
dtype = torch.float
ltype = torch.long # entropy

## Hyperparameter 셋팅

In [ ]:
random_seed = 42

batch_size = 64
validation_ratio = 0.05

num_epochs = 60
learning_rate = 0.0015
betas = (0.9, 0.999)
weight_decay = 1e-4
T_max = 50

num_workers = 8

## Randomness 제어

In [ ]:
# 1. Pytorch
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU

# 2. CuDNN
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True # 원래 False 여야 랜덤 제어

# 3. Numpy
np.random.seed(random_seed)

# 4. random (for torchvision transforms)
random.seed(random_seed)

os.environ['PYTHONHASHSEED'] = str(random_seed)

# 데이터셋 준비

## CSV파일 로드

In [ ]:
# fake 1, real 0 로 사용
train_df = pd.read_csv('./face_image/face_images.csv')
train_df.head()

,path,real,fake
0,./face_image/fake/JFH50GFJUL.jpg,0,1
1,./face_image/fake/0VPS5TI60G.jpg,0,1
2,./face_image/real/61911.jpg,1,0
3,./face_image/fake/APADHGXN31.jpg,0,1
4,./face_image/fake/SJO2UL69C2.jpg,0,1


In [ ]:
print(train_df.shape)

(20000, 3)


## 커스텀 데이터 정의

In [ ]:
class FaceDataset(Dataset):
  def __init__(self, image_label, transforms) :
    self.df = image_label
    self.transforms = transforms
        
  def __len__(self) -> int:
    return self.df.shape[0]

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
      
    image_dir = self.df.iloc[index, ]['path']
    image_id = self.df.iloc[index, ]['fake'].astype(np.int64)
    
    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    target = torch.as_tensor(image_id, dtype=torch.long)

    if self.transforms is not None :
      image = self.transforms(image=image)['image']
    
    #image = image/255.0

    return image, target

class TestDataset(Dataset):
  def __init__(self, image, transforms) :
    self.image = image
    self.transforms = transforms
        
  def __len__(self) -> int:
    return len(self.image)

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
    
    image_name = self.image[index]
    image_dir = './face_image/test_v1.1/' + image_name

    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    
    if self.transforms is not None :
      image = self.transforms(image=image)['image']

    #image = image/255.0

    return image_name, image

## 이미지 어그멘테이션

In [ ]:
transforms_tr = A.Compose([
    A.Resize(256, 256),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.GaussNoise(p=0.1),
    A.GaussianBlur(blur_limit=3, p=0.05),
    A.HorizontalFlip(),
    A.OneOf([
        A.RandomBrightnessContrast(), 
        A.FancyPCA(), 
        A.HueSaturationValue(),
    ], p=0.7),
    A.ToGray(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=10, border_mode=cv2.BORDER_CONSTANT, p=0.5),
    A.CenterCrop(156, 156),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

transforms_val = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(156, 156),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), 
])

## 데이터셋 분할

In [ ]:
#train : valid = 9 : 1
train, valid = train_test_split(train_df, test_size=validation_ratio)

print(f'Train Set dim : (%d, %d)' % (train.shape))
print(f'Valid Set dim : (%d, %d)' % (valid.shape))

Train Set dim : (19000, 3)
Valid Set dim : (1000, 3)


In [ ]:
tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

train_loader = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
print("train set size :",len(tr_dataset))
print("valid set size :",len(val_dataset))

train set size : 19000
valid set size : 1000


# 학습

In [ ]:
EARLY_STOPPING_EPOCH = 7
n_splits = 4
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

best_models = {}
fold_results = {}

for fold, (train_ids, test_ids) in enumerate(kfold.split(train_df)):
  print(f' ---------------------- Fold %d --------------------------------------------' % (fold+1) )
  
  torch.cuda.empty_cache()

  train = train_df.iloc[train_ids]
  valid = train_df.iloc[test_ids]

  tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

  # Create model
  model = InceptionResnetV1(pretrained=None, classify=True, num_classes=2, dropout_prob=0.6)
  model.to(device)

  optimizer = optim.RAdam(model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
  lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=T_max)

  # early stopping
  valid_early_stop = 0
  valid_best_loss = float('inf')
  since = time.time()

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    model.train()
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/batch_size * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 100 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    model.eval()
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)

      with torch.no_grad():
        scores = model(images)
        loss = F.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/batch_size * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0  
      # new best model save (valid 기준)
      best_model = model
      best_models[fold] = best_model
      # 저장
      path = './'
      torch.save(best_model.state_dict(), f'{path}fold{fold}model{val_mean_acc:2.2f}_epoch_{e}.pth')
      # update fold result
      fold_results[fold] = {"train_mean_acc" : train_mean_acc, 
                            "train_mean_loss" : train_mean_loss, 
                            "val_mean_acc" : val_mean_acc, 
                            "val_mean_loss" : val_mean_loss,
                            "epoch" : e}

    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:  # patience
        print("EARLY STOPPING!!")
        break

    lr_sched.step()

# **추론**

In [ ]:
submission = pd.read_csv("./face_image/submission_v1.1.csv")
submission.head()

,image,label
0,test14200.jpg,NaN
1,test12178.jpg,NaN
2,test12713.jpg,NaN
3,test13712.jpg,NaN
4,test11739.jpg,NaN


In [ ]:
test_dataset = TestDataset(submission['image'], transforms_val)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
scores_result = []

for fold in range(n_splits):
  model = best_models[fold]
  #path = './model/'
  #val_mean_acc = fold_results[fold]['val_mean_acc']
  #e = fold_results[fold]['epoch']
  #mypath = f'{path}fold{fold}model{val_mean_acc:2.2f}_epoch_{e}.pth'
  #checkpoint = torch.load(mypath)
  #model.load_state_dict(checkpoint)
  model.to(device)
  model.eval()

  predictions = []
  files = []
  score_list = []

  with torch.no_grad():
    for img_names, images in test_loader:
      images = images.to(device=device, dtype=dtype)
      scores = model(images)
      _, preds = scores.max(dim=1)
      
      files.extend(img_names)
      predictions.extend(preds.squeeze(0).detach().cpu().numpy())
      score_list.extend(scores.squeeze(0).detach().cpu().numpy())
  scores_result.append(score_list)

In [ ]:
myresult = torch.tensor(scores_result)
print(myresult.shape)
myresult = F.softmax(myresult, dim=2)
print(myresult.shape)
myresult = torch.sum(myresult, dim=0)
print(myresult.shape)

_, preds = myresult.max(dim=1)

ensemble_sub = pd.DataFrame(columns=submission.columns)
ensemble_sub['image'] = files
ensemble_sub['label'] = preds

csvfilename = "./submission_group2.csv"
ensemble_sub.to_csv(csvfilename, index=False)

In [ ]:
from google.colab import files
files.download(csvfilename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>